# Chapter 1. 约束规划

约束规划问题是在一组约束条件的限制下，求一目标函数最大或最小的问题。

## 线性规划

线性规划问题是约束与目标均线性的问题，其解必然在约束边界上，且（忽略精度问题）
现有算法能轻松解算出正确解。

### 基础示例
例 1 某机床厂生产甲、乙两种机床，每台销售后的利润分别为 4000 元与 3000 元。
生产甲机床需用A、B 机器加工，加工时间分别为每台 2 小时和 1 小时；生产乙机床需用
A、B、C 三种机器加工，加工时间为每台各一小时。若每天可用于加工的机器时数分别为
A 机器 10 小时、B 机器 8 小时和 C 机器 7 小时，问该厂应生产甲、乙机床各几台，
才能使总利润最大？

我们设该厂生产 $x_1$ 台甲机床和 $x_2$ 乙机床时总利润最大，则 $x_1, x_2$ 应满足

$$
\begin{align}
\max\qquad & z=4x_1+3x_2 \\ 
\text{s.t.}\qquad & \left\{
    \begin{array}{l}
    2x_1+x_2 \le 10 \\
    x_1+x_2 \le 8 \\
    x_2 \le 7 \\
    x_1,x_2 \ge 0 \\
    \end{array}
\right.\\
\end{align}
$$

这里变量 $x_1, x_2$ 称之为决策变量，(1) 式被称为问题的目标函数，(2) 中的几个不等式
是问题的约束条件。由于上面的目标函数及约束条件均为线性函数，故被称为线性规划问题。

### 数值计算标准型
我们采用 MATLAB/Scipy 标准型
$$
\begin{align*}
\min_x\qquad & c^Tx \\ 
\text{s.t.}\qquad & \left\{
    \begin{align}
    &A_{ub}x \le b_{ub}\qquad \\
    &A_{eq}x = b_{eq}\qquad \\
    &l \le x \le u \qquad
    \end{align}
\right.\\
\end{align*}
$$

碎碎念：尽管数学上 (3) 可以合并到 (1) 中，列式上也是如此，但是为了计算效率最好将 (3) 提取出来

Reference: [linprog | Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html)

### 求解器
我们使用 `scipy.optimize.linprog` 作为求解器。

Requirement: `scipy`

函数原型： `linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=(0, None), ...)`

其中 `c, A_ub, b_ub, A_eq, b_eq` 为标准型对应部分，`bounds` 应为 `list(zip(l, r))`

下面是例 1 的求解示例

In [2]:
# linprog(2)
from scipy.optimize import linprog

c=[-4, -3] # min of negative
A_ub=[[2,1],[1,1]]
b_ub=[10,8]
A_eq=None
b_eq=None
bounds=[(0, None), (0, 7)]

# result should be x*=[2,6], z*=-26
linprog(c, A_ub, b_ub, A_eq, b_eq, bounds)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -26.0
              x: [ 2.000e+00  6.000e+00]
            nit: 2
          lower:  residual: [ 2.000e+00  6.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-1.000e+00 -2.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

## 小技巧
### 分段函数
例： $\min |x_1|+|x_2|\ s.t.\ Ax\le b$

常规思路是分类讨论。但是利用线性规划的解在边界上的特性，可以构造
$u_i=\frac{x_i+|x_i|}2, v_i=\frac{|x_i|-x_i}2$，
则任意 $x_i$ 被一一映射到了边界 $u_iv_i=0, u_i\ge0, v_i\ge0$ 上。因此只需在边界条件中加入
$u_i\ge0, v_i\ge0$ 即可。

代价：双倍的决策变量！

### 嵌套 min max
例： $\min_x\max_y c\begin{bmatrix} x \\ \hline y \end{bmatrix} $

做法是令内层 min/max 为决策变量。例如此题，令 $z = \max_y c\begin{bmatrix} x \\ \hline y \end{bmatrix} $，
则转换为 $\min_x z\ s.t.\ c\begin{bmatrix} x \\ \hline y \end{bmatrix}\le z$ 。

代价：多 $c$ 行数个约束。

### 互斥规划
互斥/多选一规划问题有约束 $\sum_{j=1}^nx_{ij}=1, x_{ij}=0\ \text{or}\ 1$ （注意区分 0-1 规划）。
这里约束 $x_{ij}=0\ \text{or}\ 1$ 由于线性规划解特点，以及 $\sum_{j=1}^nx_{ij}=1$ 
的存在，可以被边界 $x_{ij}\ge0$ 代替。

例如，指派问题就是一个互斥规划问题。当然指派问题建议使用匈牙利算法而非线性规划。（见最底下）

衍生技巧：存在一组或关系的约束 $A_ix\le b_i$ 时，可以构造互斥变量变量 m_i，则约束变为和关系的
$A_ix\le b_i+(1-m_i)\infty$，其中 $\infty$ 可以取合适的足够大的值。

### 整数线性规划
我的评价是，不需要学习算法，建议直接调包。

`scipy.optimize.linprog` 存在选项 `integrality`。可以 `linprog(..., integrality=[1, 0,...],)` 来
声明一个变量为整数。0,1 以外的值请参考文档。

0-1 变量？直接边界 [0,1]，再加整数约束就好了。

## 小工具
下面是一些有助于面向 CV 编程的代码片段

In [35]:
# 线性规划模板
import scipy
import numpy
from numpy import asarray as aa

class Linprog:
    '''
    fields: A_ub,b_ub,A_eq,b_eq,bounds: same as linprog param, can edit and append
    '''
    def __init__(self, 
        c,
        A_ub=None, b_ub=None,
        A_eq=None, b_eq=None,
        bounds=None,
    ):
        '''
        params:  
        `c`: the cT vector  
        `A_ub,b_ub,A_eq,b_eq,bounds`: same as linprog param
        '''
        # const
        self.c=aa(c)
        size=self.c.shape[0]
        # appendable
        self.A_ub=list(A_ub) if A_ub else []
        self.b_ub=list(b_ub) if b_ub else []
        self.A_eq=list(A_eq) if A_eq else []
        self.b_eq=list(b_eq) if b_eq else []
        # fix sized
        self.bounds=bounds if bounds else [(None, None)]*size
        self.integrality=[0]*size

    '''
    互斥规划
    @param mutex_set: 0-1 互斥变量下标集合
    '''
    def add_mutex(self, mutex_set):
        mutex_aeq=numpy.zeros_like(c)
        for vint in mutex_set:
            self.bounds[vint]=(0, None)
            mutex_aeq[vint]=1
        self.A_eq.append(mutex_aeq)
        self.b_eq.append(1)
    
    def set_int(self, index: bool | int | list[int]):
        if index==True:
            self.integrality=[1]*len(self.integrality)
            return
        try:
            index=int(index)
            self.integrality[index]=1
        except:
            for i in index:
                self.integrality[i]=1
    
    def __call__(self):
        A_ub=self.A_ub if self.A_ub else None
        b_ub=self.b_ub if self.b_ub else None
        A_eq=self.A_eq if self.A_eq else None
        b_eq=self.b_eq if self.b_eq else None
        return scipy.optimize.linprog(
            self.c, A_ub, b_ub, A_eq, b_eq, 
            self.bounds,
            integrality=self.integrality
        )

In [28]:
# 线性规划示例
c=[-4, -3] # min of negative
A_ub=[[2,1],[1,1]]
b_ub=[10,8]
A_eq=None
b_eq=None
bounds=[(0, None), (0, 7)]

lp=Linprog(c,A_ub, b_ub, bounds=bounds)

lp()

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -26.0
              x: [ 2.000e+00  6.000e+00]
            nit: 2
          lower:  residual: [ 2.000e+00  6.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-1.000e+00 -2.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [36]:
# 整数示例
c=[-40, -90] # min of negative
A_ub=[[9,7],[7,20]]
b_ub=[56,70]
bounds=[(0, None), (0, None)]

lp=Linprog(c,A_ub, b_ub, bounds=bounds)
lp.set_int(True)

lp()

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -340.0
              x: [ 4.000e+00  2.000e+00]
            nit: -1
          lower:  residual: [ 4.000e+00  2.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 6.000e+00  2.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
 mip_node_count: 1
 mip_dual_bound: -340.0
        mip_gap: 0.0

In [40]:
# 指派问题 - 线性规划法
c=[
    16, 15, 19, 22,
    17, 21, 19, 18,
    24, 22, 18, 17,
    17, 19, 22, 16,
]
lp=Linprog(c)

for i in range(4):
    lp.add_mutex([i+j*4 for j in range(4)])
    lp.add_mutex([j+i*4 for j in range(4)])

lp().x

array([ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1., -0., -0.,
       -0.,  0.,  1.])

In [38]:
# 指派问题 - 匈牙利算法
c=[
    [16, 15, 19, 22],
    [17, 21, 19, 18],
    [24, 22, 18, 17],
    [17, 19, 22, 16],
]
scipy.optimize.linear_sum_assignment(c)

(array([0, 1, 2, 3]), array([1, 0, 2, 3]))

## 非线性规划
非线性规划就十分暴力了——因为它实在太宽泛了。离散的，连续的，周期的，发散的……千奇百怪的
函数都可以归类到非线性函数内，导致没有任何函数性质能给我们用。

真正无性质的函数，就只能暴力枚举/搜索。有一定连续性和光滑性的函数可以通过一些微分法快速逼近
极值。但任何非线性规划的算法都不能确保解是最佳解。

有些情况可以额外设一两个“偏常量”出来，让问题化为线性规划，然后通过目标与这一两个常量的 2D/3D 图像
来目测找出最佳点。

### 求解器
我们可以使用 `scipy.optimize.minimize` 作为求解器。建议解完多迭代几遍。下面是示例

$$
\begin{align*}
\min\qquad & x_1^2+x_2^2+x_3^2+8 \\ 
\text{s.t.}\qquad & \left\{
    \begin{align*}
    &x_1^2-x_2+x_3^2\ge0 \\
    &x_1+x_2^2+x_3^3\le20 \\
    &-x_1-x_2^2+2=0 \\
    &x_2+2x_3^2=3 \\
    &x_1,x_2,x_3\ge0\\
    \end{align*}
\right.\\
\end{align*}
$$

In [82]:
from scipy.optimize import minimize, NonlinearConstraint, LinearConstraint

minimize(
    fun=lambda v: v[0]*v[0]+v[1]*v[1]+v[2]*v[2]+8,
    x0=[0,0,0],
    constraints=[
        NonlinearConstraint(lambda v:v[0]*v[0]-v[1]+v[2]*v[2], 0, numpy.inf),
        NonlinearConstraint(lambda v:v[0]+v[1]*v[1]+v[2]**3, -numpy.inf, 20),
        NonlinearConstraint(lambda v:-v[0]-v[1]*v[1]+2, 0, 0),
        NonlinearConstraint(lambda v:v[1]+2*v[2]*v[2], 3, 3),
        LinearConstraint(numpy.eye(3),numpy.zeros((3)), numpy.inf)
    ],
)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 10.651091840572583
       x: [ 5.522e-01  1.203e+00  9.478e-01]
     nit: 15
     jac: [ 1.104e+00  2.407e+00  1.896e+00]
    nfev: 71
    njev: 15

## TODO
- 灵敏度分析
- 蒙特卡洛法